In [ ]:
from cassandra.cluster import Cluster
import csv
import pandas as pd
import os
import glob
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
cluster = Cluster(['localhost'])
session = cluster.connect('nirf')

In [ ]:
disciplines = [
    'Architecture',
    'College',
    'Dental',
    'Engineering',
    'Law',
    'Management',
    'Medical',
    'Overall',
    'Pharmacy',
    'Research',
    'University'
]

In [ ]:
nirf_score_insert_stmt = session.prepare("""
    INSERT INTO nirf_score_by_faculty (
        faculty_name, discipline, year, rank, aggregated_score, 
        teaching_learning_and_resources, research_and_professional_practice, 
        graduation_outcomes, outreach_and_inclusivity, peer_perception
    ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
""")

faculty_list_insert_stmt = session.prepare("""
    INSERT INTO faculties_list (faculty_name) VALUES (?)
""")

nirf_score_change_prior_insert_stmt = session.prepare("""
    INSERT INTO nirf_score_change_by_prior_perception (
        discipline, rounded_prior_perception, faculty_improved, change_in_teaching_learning_and_resources,
        change_in_research_and_professional_practice, change_in_graduation_outcomes,
        change_in_outreach_and_inclusivity, change_in_peer_perception
    ) VALUES (?, ?, ?, ?, ?, ?, ?, ?)
""")

nirf_score_change_post_insert_stmt = session.prepare("""
    INSERT INTO nirf_score_change_by_post_perception (
        discipline, rounded_post_perception, faculty_improved, change_in_teaching_learning_and_resources,
        change_in_research_and_professional_practice, change_in_graduation_outcomes,
        change_in_outreach_and_inclusivity, change_in_peer_perception
    ) VALUES (?, ?, ?, ?, ?, ?, ?, ?)
""")

get_all_information_for_faculty = session.prepare("""
    SELECT * FROM nirf_score_by_faculty WHERE faculty_name=? AND discipline=?
""")

In [ ]:
# This cell is intended to clean the data to prevent issues when importing into Cassandra.
# Specifically, we check to make sure that we do not have duplicate institute names within the same discipline and year.
# Additionally, we check to make sure that each entry has a valid rank.

script_dir = %pwd
base_path = os.path.join(script_dir, 'Dataset')

for dirpath, dirnames, files in os.walk(base_path):
    if dirpath == base_path:
        continue

    for file in glob.glob(os.path.join(dirpath, '*Ranking_*.csv')):
        df = pd.read_csv(file)
        duplicate_names = df[df.duplicated('Institute Name', keep=False)]['Institute Name'].unique()
        for name in duplicate_names:
            df.loc[df['Institute Name'] == name, 'Institute Name'] = df['Institute Name'] + ' ' + df['City']
        df['Rank'] = df['Rank'].astype(str).str.extract('(\d+)')[0].fillna(-1).astype(int)
        df.to_csv(file, index=False)

In [ ]:
script_dir = %pwd
base_path = os.path.join(script_dir, 'Dataset')

for dirpath, dirnames, files in os.walk(base_path):
    if dirpath == base_path:
        continue

    for file in glob.glob(os.path.join(dirpath, '*Ranking_*.csv')):
        filename = os.path.basename(file)
        faculty, year_part = filename.split('Ranking_')
        year = year_part.split('.')[0]

        nirf_score_specific_insert_stmt = session.prepare(f"""
        INSERT INTO nirf_score_by_faculty_for_{faculty} (
            faculty_name, year, rank, aggregated_score, 
            teaching_learning_and_resources, research_and_professional_practice, 
            graduation_outcomes, outreach_and_inclusivity, peer_perception
        ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
        """)
        
        with open(file, 'r') as csvfile:
            reader = csv.reader(csvfile)
            next(reader, None)
            for row in reader:
                session.execute(nirf_score_insert_stmt, (row[1], faculty, int(year), int(row[5]), float(row[4]), float(row[6]), float(row[7]), float(row[8]), float(row[9]), float(row[10])))
                session.execute(faculty_list_insert_stmt, (row[1],))
                session.execute(nirf_score_specific_insert_stmt, (row[1], int(year), int(row[5]), float(row[4]), float(row[6]), float(row[7]), float(row[8]), float(row[9]), float(row[10])))

In [ ]:
rows = session.execute("""
    SELECT * FROM faculties_list;
""")

for row in rows:
    faculty_name = row[0]
    for discipline in disciplines:
        information = session.execute(get_all_information_for_faculty, [faculty_name, discipline])
        if information:
            df = pd.DataFrame(list(information))
            df = df.round(2)
            num_rows, _ = df.shape
    
            nirf_score_change_prior_specific_insert_stmt = session.prepare(f"""
            INSERT INTO nirf_score_change_by_prior_perception_for_{discipline} (
                rounded_prior_perception, faculty_improved, change_in_teaching_learning_and_resources,
                change_in_research_and_professional_practice, change_in_graduation_outcomes,
                change_in_outreach_and_inclusivity, change_in_peer_perception
            ) VALUES (?, ?, ?, ?, ?, ?, ?)
            """)
    
            for j in range (0, num_rows - 1):
                n = df.iloc[j]
                np = df.iloc[j+1]
                delta_tlr = np[9] - n[9]
                delta_rpp = np[8] - n[8]
                delta_go = np[4] - n[4]
                delta_oi = np[5] - n[5]
                delta_pp = np[6] - n[6]
                session.execute(nirf_score_change_prior_insert_stmt, (discipline, int(n[6]), delta_pp>0, delta_tlr, delta_rpp, delta_go, delta_oi, delta_pp))
                session.execute(nirf_score_change_post_insert_stmt, (discipline, int(np[6]), delta_pp>0, delta_tlr, delta_rpp, delta_go, delta_oi, delta_pp))
                session.execute(nirf_score_change_prior_specific_insert_stmt, (int(n[6]), delta_pp>0, delta_tlr, delta_rpp, delta_go, delta_oi, delta_pp))

## Dataset As a Whole

In [ ]:
def format_column_name(col):
    return ' '.join(word.capitalize() for word in col.split('_'))

In [ ]:
# Query that retrieves data to be presented in the following cells.
result_set = session.execute("""
    SELECT * FROM nirf_score_change_by_prior_perception;
""")
df = pd.DataFrame(list(result_set))

In [ ]:
score_columns = ['change_in_teaching_learning_and_resources', 'change_in_research_and_professional_practice', 'change_in_graduation_outcomes', 'change_in_outreach_and_inclusivity']

for col in score_columns:
    sns.scatterplot(data=df, x='rounded_prior_perception', y=col)
    col_name = format_column_name(col)
    plt.title(f'Peer Perception vs {col_name}')
    plt.xlabel('Rounded Peer Perception')
    plt.ylabel(col_name)
    plt.show()

In [ ]:
score_columns = ['change_in_teaching_learning_and_resources', 'change_in_research_and_professional_practice', 'change_in_graduation_outcomes', 'change_in_outreach_and_inclusivity']

for col in score_columns:
    sns.scatterplot(data=df, x='change_in_peer_perception', y=col)
    col_name = format_column_name(col)
    plt.title(f'Peer Perception vs {col_name}')
    plt.xlabel('Change in Peer Perception')
    plt.ylabel(col_name)
    plt.show()

In [ ]:
df['final_peer_perception'] = df['rounded_prior_perception'] + df['change_in_peer_perception']
factors = ['change_in_teaching_learning_and_resources', 'change_in_research_and_professional_practice', 
           'change_in_graduation_outcomes', 'change_in_outreach_and_inclusivity']

for factor in factors:
    plt.figure(figsize=(10, 6))
    
    sns.scatterplot(data=df, x='rounded_prior_perception', y=factor, color='blue', label='Original')
    sns.scatterplot(data=df, x='final_peer_perception', y=factor, color='red', label='Final')

    for index, row in df.iterrows():
        plt.plot([row['rounded_prior_perception'], row['final_peer_perception']],
                 [row[factor], row[factor]], 'gray', linestyle='--')

    factor_name = format_column_name(factor)
    plt.title(f'Change in {factor_name} with Peer Perception')
    plt.xlabel('Peer Perception')
    plt.ylabel(f'Change in {factor_name}')
    plt.legend()
    plt.show()


## Dataset By Prior Perception Score

In [ ]:
# Query that retrieves data to be presented in the following cells.
# ALLOW FILTERING is used to because this is an individual query in which we want to receive data from all disciplines.

result_set = session.execute("""
    SELECT * FROM nirf_score_change_by_prior_perception WHERE rounded_prior_perception=45 ALLOW FILTERING;
""")
df = pd.DataFrame(list(result_set))
df = df.round(2)
df = df.sort_values(by='change_in_peer_perception', ascending=True)
df

In [ ]:
plt.figure(figsize=(10, 6))

sns.scatterplot(data=df, x='change_in_peer_perception', y='change_in_teaching_learning_and_resources', color='blue', label='TLR')
sns.scatterplot(data=df, x='change_in_peer_perception', y='change_in_research_and_professional_practice', color='green', label='RPP')
sns.scatterplot(data=df, x='change_in_peer_perception', y='change_in_graduation_outcomes', color='red', label='GO')
sns.scatterplot(data=df, x='change_in_peer_perception', y='change_in_outreach_and_inclusivity', color='purple', label='OI')

plt.title('Change in Scores vs Change in Peer Perception (Scatter Plot)')
plt.xlabel('Change in Peer Perception')
plt.ylabel('Scores')
plt.legend()
plt.show()


In [ ]:
plt.figure(figsize=(10, 6))

plt.plot(df['change_in_peer_perception'], df['change_in_teaching_learning_and_resources'], color='blue', label='TLR')
plt.plot(df['change_in_peer_perception'], df['change_in_research_and_professional_practice'], color='green', label='RPP')
plt.plot(df['change_in_peer_perception'], df['change_in_graduation_outcomes'], color='red', label='GO')
plt.plot(df['change_in_peer_perception'], df['change_in_outreach_and_inclusivity'], color='purple', label='OI')

plt.title('Change in Scores vs Change in Peer Perception (Line Plot)')
plt.xlabel('Change in Peer Perception')
plt.ylabel('Scores')
plt.legend()
plt.show()


Shutdown

In [ ]:
cluster.shutdown()